## Five Rules of Effective Prompt Engineering
1. Clear instructions improve relevance.
2. Examples stabilize the response.
3. A defined answer format = a predictable outcome.
4. Breaking work into steps = better and more complete solutions.
5. Testing and verification = safety and correctness.

In [12]:
# Import libraries
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

# Default model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

### Rule 1 - Clear instructions

In [13]:
# Bad prompt — vague, no role or expectations
bad_prompt = "Write a function in Python."
print("=== Bad prompt ===")
print(llm.invoke(bad_prompt).content)

# Good prompt — clearly defined role and expectations
good_prompt = """You are an expert Python programmer.
Write a Python function that takes a list of integers
and returns a new list containing only even numbers.
Add a unit test in pytest."""
print("\n=== Correct prompt ===")
print(llm.invoke(good_prompt).content)

=== Bad prompt ===
Sure! What kind of function would you like to create in Python? Please provide some details about its purpose, input parameters, and expected output so I can assist you better.

=== Correct prompt ===
Certainly! Below is a Python function that takes a list of integers and returns a new list containing only the even numbers. I've also included a unit test using the `pytest` framework.

### Python Function

```python
def filter_even_numbers(numbers):
    """Return a new list containing only the even numbers from the input list."""
    return [num for num in numbers if num % 2 == 0]
```

### Unit Test Using Pytest

To test the function using `pytest`, you can create a separate test file. Here’s how you would write the unit test:

```python
import pytest
from your_module import filter_even_numbers  # Replace 'your_module' with the name of your Python file without the .py extension

def test_filter_even_numbers():
    assert filter_even_numbers([1, 2, 3, 4, 5, 6]) == [2, 

### Rule 2 - Use examples

In [14]:
# Task: Generate tags based on the content of the company's website
# Zero-shot
zero_shot = """Podaj tagi opisujące firmę na podstawie tekstu strony:
Firma Lego produkuje zabawki dla dzieci."""
print("=== Zero-shot ===")
print(llm.invoke(zero_shot).content)

# One-shot
one_shot = """Enter up to three tags that describe your company based on the text on your page.
Example:
Text: Lego manufactures building blocks for children.
Tags: toys, building blocks, children

Now:
Text: Nike manufactures athletic clothing and shoes.
Tags:"""
print("\n=== One-shot ===")
print(llm.invoke(one_shot).content)

# Few-shot
few_shot = """Enter up to three tags describing your company based on the page text.

Example 1:
Text: Lego manufactures building blocks for children.
Tags: toys, building blocks, children

Example 2:
Text: Nike manufactures sportswear and shoes.
Tags: sports, clothing, footwear

Now:
Text: Tesla manufactures electric cars and energy storage systems.
Tags:"""
print("\n=== Few-shot ===")
print(llm.invoke(few_shot).content)


=== Zero-shot ===
Oto propozycje tagów opisujących firmę Lego na podstawie podanego tekstu:

1. #Lego
2. #Zabawki
3. #DlaDzieci
4. #ProducentZabawek
5. #Kreatywność
6. #Zabawa
7. #Edukacja
8. #RozwójDziecka
9. #Klocki
10. #ZabawkiKreatywne

=== One-shot ===
athletic clothing, shoes, sports

=== Few-shot ===
electric vehicles, energy storage, technology


In [15]:
prompt = """Create a list of 5 advertising slogans for a coffee shop.
Use the style from the example slogans below:
"We'll get you coffee on the table!"
"Coffee fast, cheap, and to the point!"
"""
print(llm.invoke(prompt).content)

1. "Sip, savor, and start your day right!"
2. "Brewed for you, happiness in every cup!"
3. "Where every cup tells a story!"
4. "Your daily grind, made divine!"
5. "Coffee that fuels your passion!"


In [16]:
prompt = """Create a list of 5 advertising slogans for your coffee shop.
Adopt the style from the example slogans below:
"Our coffee is like a stroll in the Garden of Eden on a sunny, joyful day."
"Drinking our coffee will make you feel as if an angel is dancing on your tongue."
"""
print(llm.invoke(prompt).content)

1. "Sip our coffee and let your senses waltz through a blissful sunrise in a serene meadow."
2. "Our brews wrap you in a warm hug from the universe, igniting your spirit with every sip."
3. "Indulge in our coffee, where each cup is a ticket to a dreamy escape beneath a starlit sky."
4. "Taste our coffee and experience the sweet symphony of flavors orchestrating a celebration on your palate."
5. "With every sip of our coffee, feel the sun rise in your heart, filling your day with radiant joy."


### Rule 3 - define response format
input and output validation using the Pydantic library

In [17]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# Output schema definition (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="The city covered by the guide")
    summary: str = Field(..., description="A short description of the city (2–3 sentences)")
    must_do: List[str] = Field(..., description="3-5 To-Do List")

# LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a travel expert. Answer concisely."),
    ("user", "Create a quick guide to {city} for a {days}-long visit.")
])

chain = prompt | structured_llm

# Pydantic input validation + chain call
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Wrocław", days=2)

result: CityGuide = chain.invoke(req.model_dump())
print(result)
print(result.model_dump_json(indent=2))

city='Wrocław' summary='Wrocław, a vibrant city in western Poland, is known for its stunning architecture, rich history, and lively cultural scene. The city is famous for its picturesque Market Square, charming canals, and the unique dwarf statues scattered throughout the streets.' must_do=['Explore the Market Square and admire the Gothic-style Town Hall.', 'Visit the Centennial Hall, a UNESCO World Heritage site.', 'Take a stroll along the Odra River and enjoy the scenic views.', 'Discover the Wrocław Cathedral on Ostrów Tumski island.', 'Search for the Wrocław dwarfs scattered around the city.']
{
  "city": "Wrocław",
  "summary": "Wrocław, a vibrant city in western Poland, is known for its stunning architecture, rich history, and lively cultural scene. The city is famous for its picturesque Market Square, charming canals, and the unique dwarf statues scattered throughout the streets.",
  "must_do": [
    "Explore the Market Square and admire the Gothic-style Town Hall.",
    "Visit 

### Rule 4 - Break down complex tasks into steps

#### Preparing a trip plan

In [18]:
# Bad prompt — everything at once
bad_prompt = """Prepare a three‑day sightseeing plan for Poznań with a budget of 300 euros,
including attractions, restaurants, transport, and maps."""
print("=== Correct prompt ===")
print(llm.invoke(bad_prompt).content[:600], "...")

# Good prompt — step by step
good_step1 = "List the most important cultural attractions in Poznań with opening hours."
step1 = llm.invoke(good_step1).content
print("\n=== Correct prompt — step 1 ===")
print(step1[:600], "...")

good_step2 = f"Based on this list, create a sightseeing plan for 3 days, max. 4 attractions per day. Attractions: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Correct prompt — step 2 ===")
print(step2[:600], "...")


=== Correct prompt ===
Here's a three-day sightseeing plan for Poznań, Poland, with a budget of 300 euros. This plan includes attractions, restaurants, transport, and maps.

### Day 1: Explore the Old Town

**Morning:**
- **Breakfast:** Start your day with breakfast at **Café La Ru** (approx. €5).
- **Attraction:** Visit the **Old Market Square** (Stary Rynek) to see the colorful townhouses and the famous **Town Hall** (free entry).
- **Attraction:** Watch the **Goats of Poznań** at noon (free).

**Lunch:**
- **Restaurant:** Enjoy lunch at **Ratuszova** (approx. €10).

**Afternoon:**
- **Attraction:** Visit the **Na ...

=== Correct prompt — step 1 ===
Poznań, a vibrant city in Poland, is rich in history and culture. Here are some of the most important cultural attractions along with their typical opening hours. Please note that hours may vary, so it's always a good idea to check the official websites or contact the venues directly for the most current information.

1. **Old Market Squ

#### Step‑by‑step article generation

In [19]:
ARTICLE_SUBJECT = "What competencies will be most valuable for programmers in 10 years?"

plan = f"Prepare a 3-point plan for an article on the topic: {ARTICLE_SUBJECT}."
planGenerated = llm.invoke(plan).content
print(planGenerated, "...")

### 3-Point Plan for Article: "What Competencies Will Be Most Valuable for Programmers in 10 Years?"

#### 1. **Adaptability to Emerging Technologies**
   - **Overview**: Discuss the rapid pace of technological advancement and the importance of being adaptable. Highlight how programmers will need to stay current with trends such as artificial intelligence, quantum computing, and blockchain.
   - **Key Competencies**: Emphasize skills in learning new programming languages, frameworks, and tools quickly. Include the importance of understanding and integrating new technologies into existing systems.
   - **Future Outlook**: Explore how adaptability will not only enhance individual career prospects but also contribute to organizational resilience in a fast-evolving tech landscape.

#### 2. **Collaboration and Communication Skills**
   - **Overview**: Examine the shift towards more collaborative and interdisciplinary work environments in software development. As projects become more complex

In [20]:
MAX_WORDS_IN_PARAGRAPH = 200

prevParagraphContent = ""

for i in range(1,4):
    paragraphPrompt = f"Prepare only the content of the {i}th paragraph of the article: {ARTICLE_SUBJECT}. The maximum text length is {MAX_WORDS_IN_PARAGRAPH}. Generate only the text of one paragraph. Plan of the entire article:\n{planGenerated}. Content of the previous paragraphs: {prevParagraphContent}."
    paragraphContent = llm.invoke(paragraphPrompt).content
    prevParagraphContent += paragraphContent + "\n"
    print(f"Akapit {i}.", paragraphContent, "\n\n")

Akapit 1. In the rapidly evolving tech landscape, adaptability to emerging technologies will be one of the most valuable competencies for programmers in the next decade. As advancements in fields like artificial intelligence, quantum computing, and blockchain continue to reshape the industry, programmers must be prepared to learn and integrate new programming languages, frameworks, and tools swiftly. This ability to stay current with technological trends will not only enhance individual career prospects but also bolster organizational resilience, ensuring that teams can effectively navigate the complexities of an ever-changing digital environment. 


Akapit 2. The shift towards more collaborative and interdisciplinary work environments in software development underscores the growing importance of collaboration and communication skills. As projects become increasingly complex, programmers will need to work effectively in teams and engage with non-technical stakeholders to convey technic

### Rule 5 - Test and verify results

In [21]:
from langchain_classic.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result = evaluator.evaluate_strings(
    prediction="Capital of Poland is Warsaw",
    reference="something very different"
)

print(result)


{'score': 0.27325426536671193}
